In [2]:
import pandas as pd

In [3]:
prefix = './data/stanfordSentimentTreebank/'

# Load all dataframes
sentences_df = pd.read_csv(prefix + 'datasetSentences.txt', sep='\t')
dictionary_df = pd.read_csv(prefix + 'dictionary.txt', sep='|', names=['phrase', 'phrase ids'])
sentiment_labels_df = pd.read_csv(prefix + 'sentiment_labels.txt', sep='|')
dataset_split_df = pd.read_csv(prefix + 'datasetSplit.txt', sep=',')

# Merge all dataframes
sentiment_data = (sentences_df
                  .merge(dictionary_df, left_on='sentence', right_on='phrase', how='left')
                  .merge(sentiment_labels_df, on='phrase ids', how='left')
                  .merge(dataset_split_df, on='sentence_index', how='left'))

sentiment_data.head()

,sentence_index,sentence,phrase,phrase ids,sentiment values,splitset_label
0,1,The Rock is destined to be the 21st Century 's...,The Rock is destined to be the 21st Century 's...,226166.0,0.69444,1
1,2,The gorgeously elaborate continuation of `` Th...,The gorgeously elaborate continuation of `` Th...,226300.0,0.83333,1
2,3,Effective but too-tepid biopic,Effective but too-tepid biopic,13995.0,0.51389,2
3,4,If you sometimes like to go to the movies to h...,If you sometimes like to go to the movies to h...,14123.0,0.73611,2
4,5,"Emerges as something rare , an issue movie tha...","Emerges as something rare , an issue movie tha...",13999.0,0.86111,2


In [4]:
# Save dataframe to pickle
sentiment_data.to_pickle('./data/sentiment_data.pkl')